# PCA-LDA DATASET ECOLI

Notebook ini bertujuan untuk menganalisis dataset Ecoli yang tersimpan di database MySQL. Dataset Ecoli berisi beberapa fitur numerik yang menggambarkan karakteristik protein, dan sebuah label kelas yang menunjukkan lokasi subselular protein tersebut (misalnya cp, im, om, dll).

Analisis ini menggunakan dua metode reduksi dimensi:

Principal Component Analysis (PCA)

Linear Discriminant Analysis (LDA)

Keduanya akan digunakan untuk memproyeksikan data berdimensi tinggi (7 fitur) ke dalam ruang dua dimensi, sehingga distribusi data antar kelas dapat divisualisasikan.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sqlalchemy import create_engine
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

# === KONEKSI KE MYSQL ===
engine = create_engine("mysql+pymysql://root:@localhost/ecoli")

# === AMBIL DATA ===
df = pd.read_sql("SELECT * FROM ecoli", engine)
print(df.head())

# === PISAHKAN FITUR DAN LABEL ===
X = df[["feature1","feature2","feature3","feature4","feature5","feature6","feature7"]].values
y_raw = df["class_label"].values

# encode label
le = LabelEncoder()
y = le.fit_transform(y_raw)
target_names = le.classes_

# Standarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(X)

# === Distribusi kelas sebelum balancing ===
unique, counts = np.unique(y, return_counts=True)
dist_before = dict(zip(le.classes_, counts))
print("Distribusi kelas sebelum balancing:", dist_before)

plt.figure(figsize=(8,5))
plt.bar(dist_before.keys(), dist_before.values(), color="skyblue")
for i, val in enumerate(dist_before.values()):
    plt.text(i, val+2, str(val), ha='center')
plt.title("Distribusi Kelas Sebelum Balancing")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Data")
plt.show()

# === PCA DATA ASLI ===
pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

plt.figure(figsize=(7,5))
colors = ["navy","turquoise","darkorange","green","red","purple","brown","gray"]
for color, i, target_name in zip(colors, range(len(target_names)), target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=0.8, label=target_name)
plt.legend(loc="best")
plt.title("PCA of Original Ecoli dataset")
plt.show()

# === SMOTE untuk penyeimbangan data ===
smote = SMOTE(random_state=42, k_neighbors=1)
X_res, y_res = smote.fit_resample(X, y)

# Distribusi kelas sesudah balancing
unique_res, counts_res = np.unique(y_res, return_counts=True)
dist_after = dict(zip(le.inverse_transform(unique_res), counts_res))
print("Distribusi kelas sesudah balancing:", dist_after)

plt.figure(figsize=(8,5))
plt.bar(dist_after.keys(), dist_after.values(), color="lightgreen")
for i, val in enumerate(dist_after.values()):
    plt.text(i, val+2, str(val), ha='center')
plt.title("Distribusi Kelas Sesudah Balancing (SMOTE)")
plt.xlabel("Kelas")
plt.ylabel("Jumlah Data")
plt.show()

# === PCA DATA BALANCED ===
X_r_bal = pca.fit(X_res).transform(X_res)
is_synthetic = np.array([False]*len(y) + [True]*(len(X_res)-len(y)))

plt.figure(figsize=(7,5))
for color, i, target_name in zip(colors, range(len(target_names)), target_names):
    plt.scatter(
        X_r_bal[(y_res == i) & (is_synthetic == False), 0],
        X_r_bal[(y_res == i) & (is_synthetic == False), 1],
        color=color, alpha=0.6, label=f"{target_name} (original)"
    )
    plt.scatter(
        X_r_bal[(y_res == i) & (is_synthetic == True), 0],
        X_r_bal[(y_res == i) & (is_synthetic == True), 1],
        color=color, alpha=0.6, marker="x", label=f"{target_name} (synthetic)"
    )
plt.legend(loc="best", fontsize=8)
plt.title("PCA of Balanced Ecoli dataset (SMOTE)")
plt.show()

# === Fungsi evaluasi model ===
def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n=== Evaluasi {name} ===")
    print("Akurasi:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))
    
    plt.figure(figsize=(7,5))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d",
                xticklabels=target_names, yticklabels=target_names, cmap="coolwarm")
    plt.title(f"Confusion Matrix {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# === KLASIFIKASI SEBELUM BALANCING ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

evaluate_model("Naive Bayes (Sebelum Balancing)", GaussianNB(), X_train, X_test, y_train, y_test)
evaluate_model("Random Forest (Sebelum Balancing)", RandomForestClassifier(random_state=42, n_estimators=100), X_train, X_test, y_train, y_test)
evaluate_model("Bagging (Sebelum Balancing)", BaggingClassifier(random_state=42, n_estimators=100), X_train, X_test, y_train, y_test)

# === KLASIFIKASI SESUDAH BALANCING ===
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res, y_res, test_size=0.3, random_state=42, stratify=y_res)

evaluate_model("Naive Bayes (Sesudah Balancing)", GaussianNB(), X_train_res, X_test_res, y_train_res, y_test_res)
evaluate_model("Random Forest (Sesudah Balancing)", RandomForestClassifier(random_state=42, n_estimators=100), X_train_res, X_test_res, y_train_res, y_test_res)
evaluate_model("Bagging (Sesudah Balancing)", BaggingClassifier(random_state=42, n_estimators=100), X_train_res, X_test_res, y_train_res, y_test_res)


Dalam proyek ini, dataset Ecoli yang digunakan memiliki distribusi kelas yang sangat tidak seimbang. Sebagai contoh, beberapa kelas minoritas ekstrem hanya memiliki 2–5 sampel, sedangkan kelas mayoritas memiliki lebih dari 100 sampel:

{'cp': 143, 'im': 77, 'imL': 2, 'imS': 2, 'imU': 35, 'om': 20, 'omL': 5, 'pp': 52}

Awalnya, penyeimbangan data dicoba menggunakan ADASYN, karena metode ini dapat membuat sampel synthetic yang fokus pada titik-titik minoritas yang sulit dipelajari (hard-to-learn). Namun, ADASYN memerlukan jumlah sampel minoritas minimal 3 untuk mencari tetangga dari kelas mayoritas.

Dalam dataset ini, kelas imL dan imS hanya memiliki 2 sampel, sehingga ADASYN gagal mengeksekusi proses penyeimbangan dan menghasilkan error:

RuntimeError: Not any neighbours belong to the majority class...

Untuk mengatasi hal ini, dipilih SMOTE (Synthetic Minority Over-sampling Technique) sebagai metode penyeimbangan. SMOTE tetap bisa bekerja untuk kelas minoritas yang sangat kecil dengan cara:

Membuat sampel synthetic dengan menginterpolasi antara titik minoritas dan tetangga terdekatnya.

Memungkinkan pengaturan k_neighbors secara manual, sehingga dapat disesuaikan dengan jumlah sampel minoritas (dalam kasus ini, k_neighbors=1).

Dengan SMOTE, semua kelas minoritas, termasuk yang memiliki sangat sedikit sampel, dapat dibuatkan sampel synthetic dengan aman tanpa error, sehingga dataset menjadi lebih seimbang dan siap untuk analisis atau pemodelan berikutnya.